In [2]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from PIL import Image
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import hashlib
from collections import Counter
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.model_selection import train_test_split



In [ ]:
print("TensorFlow version:", tf.__version__)
print("GPUs detected:", tf.config.list_physical_devices('GPU'))

In [54]:

# Path to dataset (adjust this)
DATASET_PATH = "C:/Users/Ayush/ath_codee/1_plant_classification_dataset/cleaned_dataset"


In [55]:
def remove_corrupted_images(dataset_path):
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if not os.path.isdir(folder_path):
            continue
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                img = Image.open(file_path)
                img.verify()  # verify if image is readable
            except (IOError, SyntaxError):
                print(f"Removing corrupted image: {file_path}")
                os.remove(file_path)

remove_corrupted_images(DATASET_PATH)


Removing corrupted image: C:/Users/Ayush/ath_codee/1_plant_classification_dataset/cleaned_dataset\Coffee\P_20190924_100931.jpg


In [8]:
# Define constants
IMG_SIZE = (224, 224)
DATASET_PATH = "D:/GIG/Indian_Medical_Plants_dataset/Indian Medicinal Leaves Image Datasets/Medicinal Leaf dataset"
CLEANED_PATH = "cleaned_dataset"

def resize_images(dataset_path, output_path):
    os.makedirs(output_path, exist_ok=True)
    
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if not os.path.isdir(folder_path):
            continue
        
        output_folder = os.path.join(output_path, folder)
        os.makedirs(output_folder, exist_ok=True)
        
        print(f"Processing folder: {folder}...")
        
        for file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, file)
            try:
                img = Image.open(img_path).convert("RGB")
                img = img.resize(IMG_SIZE)
                img.save(os.path.join(output_folder, file))
            except Exception as e:
                print(f"Error resizing {img_path}: {e}")
        
        print(f"Finished folder: {folder}")

resize_images(DATASET_PATH, CLEANED_PATH)
print("All images resized and saved successfully.")

Processing folder: Aloevera...
Finished folder: Aloevera
Processing folder: Amla...
Finished folder: Amla
Processing folder: Amruthaballi...
Finished folder: Amruthaballi
Processing folder: Arali...
Finished folder: Arali
Processing folder: ashoka...
Finished folder: ashoka
Processing folder: Astma_weed...
Finished folder: Astma_weed
Processing folder: Badipala...
Finished folder: Badipala
Processing folder: Balloon_Vine...
Finished folder: Balloon_Vine
Processing folder: Bamboo...
Finished folder: Bamboo
Processing folder: Beans...
Finished folder: Beans
Processing folder: Betel...
Finished folder: Betel
Processing folder: Bhrami...
Finished folder: Bhrami
Processing folder: Bringaraja...
Finished folder: Bringaraja
Processing folder: camphor...
Finished folder: camphor
Processing folder: Caricature...
Finished folder: Caricature
Processing folder: Castor...
Finished folder: Castor
Processing folder: Catharanthus...
Finished folder: Catharanthus
Processing folder: Chakte...
Finished f

In [9]:

datagen = ImageDataGenerator(
    rescale=1./255,           # normalization
    validation_split=0.2      # 80/20 train/val split
)

train_gen = datagen.flow_from_directory(
    CLEANED_PATH,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    CLEANED_PATH,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 5535 images belonging to 80 classes.
Found 1339 images belonging to 80 classes.


In [10]:

def remove_duplicates(dataset_path):
    hashes = set()
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            with open(file_path, 'rb') as f:
                file_hash = hashlib.md5(f.read()).hexdigest()
            if file_hash in hashes:
                print(f"Duplicate found and removed: {file_path}")
                os.remove(file_path)
            else:
                hashes.add(file_hash)

remove_duplicates(CLEANED_PATH)


Duplicate found and removed: cleaned_dataset\Badipala\1182.jpg
Duplicate found and removed: cleaned_dataset\Badipala\1186.jpg
Duplicate found and removed: cleaned_dataset\Balloon_Vine\1330.jpg
Duplicate found and removed: cleaned_dataset\Bhrami\2214.jpg
Duplicate found and removed: cleaned_dataset\Bhrami\2222.jpg
Duplicate found and removed: cleaned_dataset\Doddpathre\180.jpg
Duplicate found and removed: cleaned_dataset\Doddpathre\250.jpg
Duplicate found and removed: cleaned_dataset\Globe Amarnath\194.jpg
Duplicate found and removed: cleaned_dataset\Globe Amarnath\202.jpg
Duplicate found and removed: cleaned_dataset\Globe Amarnath\208.jpg
Duplicate found and removed: cleaned_dataset\Globe Amarnath\214.jpg
Duplicate found and removed: cleaned_dataset\Globe Amarnath\222.jpg
Duplicate found and removed: cleaned_dataset\Globe Amarnath\250.jpg
Duplicate found and removed: cleaned_dataset\Globe Amarnath\272.jpg
Duplicate found and removed: cleaned_dataset\Globe Amarnath\276.jpg
Duplicate fou

In [41]:
CLEANED_PATH = "C:/Users/Ayush/ath_codee/1_plant_classification_dataset/cleaned_dataset"

aug_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
    validation_split=0.2
)

train_gen = aug_gen.flow_from_directory(
    CLEANED_PATH,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = aug_gen.flow_from_directory(
    CLEANED_PATH,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 4039 images belonging to 80 classes.
Found 965 images belonging to 80 classes.


In [12]:
def remove_background(image_path, save_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)
    result = cv2.bitwise_and(image, image, mask=mask)
    cv2.imwrite(save_path, result)


In [42]:
DATASET_DIR = "C:/Users/Ayush/ath_codee/1_plant_classification_dataset/cleaned_dataset"   # from previous preprocessing step
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Create ImageDataGenerator for normalization
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15  # 15% validation
)

# Train generator
train_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset='training',
    class_mode='categorical',
    shuffle=True,
    seed=42
)

# Validation generator
val_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset='validation',
    class_mode='categorical',
    shuffle=False,
    seed=42
)



Found 4291 images belonging to 80 classes.
Found 713 images belonging to 80 classes.


In [43]:

SOURCE = "C:/Users/Ayush/ath_codee/1_plant_classification_dataset/cleaned_dataset"
DEST_TEST = "C:/Users/Ayush/ath_codee/1_plant_classification_dataset/test_dataset"
TEST_SPLIT = 0.15  # 15% test

os.makedirs(DEST_TEST, exist_ok=True)

for class_name in os.listdir(SOURCE):
    class_path = os.path.join(SOURCE, class_name)
    if not os.path.isdir(class_path):
        continue

    # Create test folder for each class
    test_class_path = os.path.join(DEST_TEST, class_name)
    os.makedirs(test_class_path, exist_ok=True)

    # Select random 15% of images
    images = os.listdir(class_path)
    test_images = random.sample(images, int(len(images) * TEST_SPLIT))

    # Move images
    for img in test_images:
        shutil.move(os.path.join(class_path, img),
                    os.path.join(test_class_path, img))


In [44]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_gen = test_datagen.flow_from_directory(
    'C:/Users/Ayush/ath_codee/1_plant_classification_dataset/test_dataset',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


Found 2542 images belonging to 80 classes.


In [45]:

test_dir = 'C:/Users/Ayush/ath_codee/1_plant_classification_dataset/test_dataset'  # your directory path
for root, dirs, files in os.walk(test_dir):
    print(f"Current directory: {root}")
    print(f"Subdirectories: {dirs}")
    print(f"Files: {files}")


Current directory: C:/Users/Ayush/ath_codee/1_plant_classification_dataset/test_dataset
Subdirectories: ['Aloevera', 'Amla', 'Amruthaballi', 'Arali', 'ashoka', 'Astma_weed', 'Badipala', 'Balloon_Vine', 'Bamboo', 'Beans', 'Betel', 'Bhrami', 'Bringaraja', 'camphor', 'Caricature', 'Castor', 'Catharanthus', 'Chakte', 'Chilly', 'Citron lime (herelikai)', 'Coffee', 'Common rue(naagdalli)', 'Coriender', 'Curry', 'Doddpathre', 'Drumstick', 'Ekka', 'Eucalyptus', 'Ganigale', 'Ganike', 'Gasagase', 'Ginger', 'Globe Amarnath', 'Guava', 'Henna', 'Hibiscus', 'Honge', 'Insulin', 'Jackfruit', 'Jasmine', 'kamakasturi', 'Kambajala', 'Kasambruga', 'kepala', 'Kohlrabi', 'Lantana', 'Lemon', 'Lemongrass', 'Malabar_Nut', 'Malabar_Spinach', 'Mango', 'Marigold', 'Mint', 'Neem', 'Nelavembu', 'Nerale', 'Nooni', 'Onion', 'Padri', 'Palak(Spinach)', 'Papaya', 'Parijatha', 'Pea', 'Pepper', 'Pomoegranate', 'Pumpkin', 'Raddish', 'Rose', 'Sampige', 'Sapota', 'Seethaashoka', 'Seethapala', 'Spinach1', 'Tamarind', 'Taro', 

In [47]:
print("Training samples:", train_gen.samples)
print("Validation samples:", val_gen.samples)
print("Testing samples:", test_gen.samples)
print("Number of classes:", train_gen.num_classes)
print("Classes:", train_gen.class_indices)


Training samples: 4291
Validation samples: 713
Testing samples: 2542
Number of classes: 80
Classes: {'Aloevera': 0, 'Amla': 1, 'Amruthaballi': 2, 'Arali': 3, 'Astma_weed': 4, 'Badipala': 5, 'Balloon_Vine': 6, 'Bamboo': 7, 'Beans': 8, 'Betel': 9, 'Bhrami': 10, 'Bringaraja': 11, 'Caricature': 12, 'Castor': 13, 'Catharanthus': 14, 'Chakte': 15, 'Chilly': 16, 'Citron lime (herelikai)': 17, 'Coffee': 18, 'Common rue(naagdalli)': 19, 'Coriender': 20, 'Curry': 21, 'Doddpathre': 22, 'Drumstick': 23, 'Ekka': 24, 'Eucalyptus': 25, 'Ganigale': 26, 'Ganike': 27, 'Gasagase': 28, 'Ginger': 29, 'Globe Amarnath': 30, 'Guava': 31, 'Henna': 32, 'Hibiscus': 33, 'Honge': 34, 'Insulin': 35, 'Jackfruit': 36, 'Jasmine': 37, 'Kambajala': 38, 'Kasambruga': 39, 'Kohlrabi': 40, 'Lantana': 41, 'Lemon': 42, 'Lemongrass': 43, 'Malabar_Nut': 44, 'Malabar_Spinach': 45, 'Mango': 46, 'Marigold': 47, 'Mint': 48, 'Neem': 49, 'Nelavembu': 50, 'Nerale': 51, 'Nooni': 52, 'Onion': 53, 'Padri': 54, 'Palak(Spinach)': 55, 'Papa

In [49]:
print(df['label'].value_counts())


label
Tamarind          67
Tulsi             67
Curry             64
Palak(Spinach)    57
Doddpathre        53
                  ..
Sapota            15
Raddish           15
Turmeric          14
Lemongrass         2
Pepper             2
Name: count, Length: 80, dtype: int64


In [37]:
counts = df['label'].value_counts()
valid_labels = counts[counts >= 2].index
df_filtered = df[df['label'].isin(valid_labels)]


In [38]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(
    df_filtered,
    test_size=0.3,
    stratify=df_filtered['label'],
    random_state=42
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df['label'],
    random_state=42
)

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [36]:
# -------------------------------
# 2. Image Data Generators
# -------------------------------
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

# -------------------------------
# 3. Create Data Iterators from DataFrames
# -------------------------------
train_gen = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='label',
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

val_gen = val_datagen.flow_from_dataframe(
    val_df,
    x_col='filename',
    y_col='label',
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_gen = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='label',
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 1280 validated image filenames belonging to 80 classes.


NameError: name 'val_df' is not defined

In [ ]:
#label is not part of file name
#in the folder, CLEANED_DATASET there are sub-folders, with the label as their names
#subfolder name = class label


In [ ]:

# -------------------------------
# 4. Build CNN using Transfer Learning (EfficientNetB0)
# -------------------------------
base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)
base_model.trainable = False  # freeze pretrained layers

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(len(train_gen.class_indices), activation='softmax')  # <-- fixed here
])

# -------------------------------
# 5. Compile the model
# -------------------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# -------------------------------
# 6. Define Callbacks
# -------------------------------
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1),
    ModelCheckpoint("best_model.h5", save_best_only=True)
]

# -------------------------------
# 7. Train the Model
# -------------------------------
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks
)

# -------------------------------
# 8. Evaluate on Test Set
# -------------------------------
test_loss, test_acc = model.evaluate(test_gen)
print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")

# -------------------------------
# 9. Save Final Model
# -------------------------------
model.save("New_plant_classifier_dataframe.h5")
print("Model saved as New_plant_classifier_dataframe.h5")

# -------------------------------
# 10. Model Summary
# -------------------------------
model.summary()

AttributeError: 'DataFrameIterator' object has no attribute 'num_classes'

In [51]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model


In [56]:

# Constants
DATASET_DIR = "C:/Users/Ayush/ath_codee/1_plant_classification_dataset/cleaned_dataset"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10
MODEL_SAVE_PATH = "efficientnetb0_plant_species.h5"

# Create a DataFrame of filepaths and labels
images = []
labels = []
for folder in os.listdir(DATASET_DIR):
    folder_path = os.path.join(DATASET_DIR, folder)
    if not os.path.isdir(folder_path):
        continue
    for fname in os.listdir(folder_path):
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            images.append(os.path.join(folder_path, fname))
            labels.append(folder)
df = pd.DataFrame({'filename': images, 'label': labels})

# Filter classes with less than 2 samples (for stratify requirement)
counts = df['label'].value_counts()
valid_labels = counts[counts >= 2].index
df = df[df['label'].isin(valid_labels)]

# Train-test split stratified by label
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df['label'],
    random_state=42
)

# ImageDataGenerators
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

num_classes = len(train_generator.class_indices)

# Build EfficientNetB0 model with transfer learning
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False  # Freeze base for initial training

inputs = tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train model
model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS
)

# Unfreeze base and fine-tune optionally
base_model.trainable = True
fine_tune_at = len(base_model.layers) - 20
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine tune model
model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=5
)

# Save model safely ensuring BatchNormalization axis param saved as int
# Since TensorFlow 2.20 / Keras 3.12 is used, direct save is safe
model.save(MODEL_SAVE_PATH)
print(f"Model saved to {MODEL_SAVE_PATH}")


Found 4673 validated image filenames belonging to 80 classes.
Found 1169 validated image filenames belonging to 80 classes.
Epoch 1/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 90s 577ms/step - accuracy: 0.2514 - loss: 3.3349 - val_accuracy: 0.4979 - val_loss: 2.3650
Epoch 2/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 98s 667ms/step - accuracy: 0.5553 - loss: 2.0323 - val_accuracy: 0.6578 - val_loss: 1.6736
Epoch 3/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 93s 630ms/step - accuracy: 0.6837 - loss: 1.4830 - val_accuracy: 0.7220 - val_loss: 1.3414
Epoch 4/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 86s 588ms/step - accuracy: 0.7443 - loss: 1.1983 - val_accuracy: 0.7579 - val_loss: 1.1456
Epoch 5/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 87s 594ms/step - accuracy: 0.7881 - loss: 1.0094 - val_accuracy: 0.7802 - val_loss: 0.9941
Epoch 6/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 87s 590ms/step - accuracy: 0.8125 - loss: 0.8799 - val_accuracy: 0.7938 - val_loss: 0.9022
Epoch 7/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 86s 586ms/step - accuracy: 0.8320 - loss: 0.7848 -

Model saved to efficientnetb0_plant_species.h5
